### Make a Crawler to get the Pictures form NMNS with Python3
http://www.nmns.edu.tw/web/learn/feature/
- [目標相簿 - goal album 01](http://www.nmns.edu.tw/web/learn/feature/photo.htm?id=13)
- [目標相簿 - goal album 02](http://www.nmns.edu.tw/web/learn/feature/photo.htm?id=14)
- [目標相簿 - goal album 03](http://www.nmns.edu.tw/web/learn/feature/photo.htm?id=15)

environment :  
Python 3.5.2 |Anaconda 4.2.0 (64-bit)| (default, Jul  5 2016, 11:41:13) [MSC v.1900 64 bit (AMD64)] on win32  

### 1. 建立環境、匯入所需模組 / set environment , import module

In [ ]:
import selenium
from selenium import webdriver
import urllib.request
import ssl

### 2. 製作時間檢查機制 / make a time check
相簿效期只有3個月，故須判斷是否仍在效期內。（上傳日期：2018-07-29）  
the album is alive for 3 months , set urlInvalidDate to check.  
album upload date: 2018-07-29

----
##### note
- [time module](https://docs.python.org/3/library/time.html)
[中文教學](http://www.runoob.com/python/python-date-time.html)
    - time.time() : seconds since 12:00am, January 1, 1970
    - time.localtime() : show local current time in struct_time
    - time.asctime() : show time in "Www Mmm dd hh:mm:ss yyyy" format  
- [datetime module](https://docs.python.org/3/library/datetime.html#module-datetime)
[中文教學01](https://www.cnblogs.com/feeland/p/4551054.html)
[中文教學02](https://blog.csdn.net/mathcompfrac/article/details/76229385)
[中文教學03](http://blog.51cto.com/leejia/1577662)
    - datetime.datetime.today() : show local current time in datetime
    - datetime.datetime.strptime() : format string to datetime
    - (date)replace() : replace date with given values

In [ ]:
import datetime

uploadDate = "2018-07-29"
aliveMonth = 3
localtime = datetime.datetime.today()

uploadDatetime = datetime.datetime.strptime(uploadDate,"%Y-%m-%d")
replaceMonth = uploadDatetime.month + aliveMonth

urlInvalidDate = uploadDatetime.replace(month = replaceMonth)
print(localtime,type(localtime))
print(urlInvalidDate,type(urlInvalidDate))

### 3. 分析網頁結構 / analysis the HTML structure
依據網頁結構，發現2種圖片網址可能所在之 xpath & attribute 對應：  
two tpyes of the HTML structure.  

type 01
<code>
    < div   
        class="swiper-slide"  
        style="  
            background-image: url("
                https://wwwadmin.nmns.edu.tw/upload/activity/1532917261goKuy5PTED.JPG");  
            width: 767px;"  
        data-swiper-slide-index="58">  
    < /div>
</code>

type 02
<code>
    < div 
         class="swiper-slide" 
         style="text-align: center; width: 669px;" 
         data-swiper-slide-index="9">
         < img
             src="https://wwwadmin.nmns.edu.tw/upload/activity/1532917211bBTZsVMFkB.JPG" 
             height="100%" 
             style="max-width:100%;">
    < /div>
</code>

In [ ]:
xpath_attribute = {
    '//div[@data-swiper-slide-index]':'style',
    '//div[@data-swiper-slide-index]/img':'src'
}

# urls to be get
url_dict = {}

### 4. 開啟瀏覽器、設定抓取天數 / open the browser , set how many album to crawl through
* the albums are separated by days

In [ ]:
driver = webdriver.Chrome(r'D:\pyLadies\2018_0805_nmnsCrawler\chromedriver.exe')
totalDays = 3

### 5. 自行撰寫方法「findURL()」以取得單一相片網址 /  
### make a method "findURL()" to get the URLs of the pictures
1. 分別以觀察到的各xpath+attribute組合檢查，並計算圖片張數以製作檔名。  
    check with the set "xpath+attribute", and set a counter for the file name
2. to get the URL of the images
	1. 以「tagContent」取出特定標籤特定屬性內全部的屬性值。  
       get all attribute contents of the certain attribute, put into "tagContent"
    2. 處理style屬性值。 / clean the "tagContent" from the attribute "style"  
       藉由實際觀察得知，如果網址在style的屬性值裡，則取出全部屬性值會出現以「\」分隔的字串資料，而且網址是最後一個屬性值。故再將tagContent以「\」分割，並取出網址部份。  
       content of attribute "style" is seperate with "\", and the url part is the last part, update the "tagContent"
    3. 接著確認目前的「tagContent」是否成功包含到網址段。  
       make sure there is the url in the "tagContent"  
       若「否」則回傳False；  
       if not, return False  
       「是」則取網址到準備好的url_dict{}檢查網址是否有重複。  
       if there is the url, check the url in url_dict{} to avoid a duplication  
       未重複就對圖片張數加一，並儲存圖片網址、製作對應檔名。  
       count for the picture, and fill the url_dict{} with the url as a key,the file name as a value

----
##### note
- string.find(value, start, end) : return start index of value in the string, or return "-1" if there is no match value.
    - [w3schools](https://www.w3schools.com/python/ref_string_find.asp)
    - [菜鳥教程](http://www.runoob.com/python3/python3-string-find.html)

In [ ]:
# method findURL() : get the clean URLs, and put in url_dict{}
def findURL(xpath,attribute):
    images = driver.find_elements_by_xpath(xpath)
    picNumber = 0 # 圖片張數計算 picture number to count how many pictures in this album

    for image in images:
        tagContent = image.get_attribute(attribute)
        
        if tagContent.find('\"') == -1: pass              
        else: 
            tagContent = tagContent.split('\"')[-1]
            # ↑split the string if it comes from attribute "style" 

        if tagContent.find("https://") == -1: return False            
        else:
            url = tagContent.split('\"')[0]
            # ↑split to get clear url from attribute "style" ????????redundant????
            if url in url_dict: pass
            else: 
                picNumber += 1
                url_dict[url] = "Day" + str(day) + "_" + str(picNumber).zfill(3) + ".jpg"
            

### 6. 主程式 / main block
1. 確認程式執行時間相簿是否仍可存取。時間比較：未來 > 現在 > 過去  
   check the InvalidDate to make sure the album is still avalible  
   *future is larger than now, now is larger than past*
2. 確認尚可存取後 / if the album is still avalible
    1. 以抓取天數（totalDays）迴圈執行相簿抓取工作（findURL()）。  
        1. 依觀察出的規律製作抓取網頁網址，並取得網頁內容。  
            for example:
            url of day 1 = http://www.nmns.edu.tw/web/learn/feature/photo.htm?id=13
            url of day 2 = http://www.nmns.edu.tw/web/learn/feature/photo.htm?id=14
            conclusion: id = 12 + day number

        2. 依觀察出的HTML結構分別試驗抓取工作，取得圖片網址與檔名對應。
    2. 避開SSL安全檢核下載檔案。  
       download the images as ".jpg" without the SSL certificate verify

----
##### note
- SSL：安全通訊協定 Secure Sockets Layer

In [ ]:
# main
if localtime > urlInvalidDate: # check the date
    print("相簿已逾期！請手動確認檔案是否仍存在。")
else:
    for day in range(1,totalDays+1): # get the url of the day
        urlDay = 'http://www.nmns.edu.tw/web/learn/feature/photo.htm?id={idNo}'.format(idNo=12+day)
        driver.get(urlDay)

        for xpath,attribute in  xpath_attribute.items(): # get the url from xpath
            if findURL(xpath,attribute) != False:
                print("Day",day,"取得網址！","目前字典長度:",len(url_dict))
            else:
                print( "注意：「", xpath, " + ", attribute, "」的組合失敗。" )
                  
    for key,value in url_dict.items(): #download images from the url,and save them with the file names
        ssl._create_default_https_context = ssl._create_unverified_context
            # ↑ to pass the SSL certificate verify
        resource = urllib.request.urlopen(key)
        output = open(value,"wb") # write binary
        output.write(resource.read())
        output.close()
    print("下載結束")

＊output of main blolck
----

注意：「 //div[@data-swiper-slide-index]  +  style 」的組合失敗。  
Day 1 取得網址！ 目前字典長度: 85  
注意：「 //div[@data-swiper-slide-index]  +  style 」的組合失敗。  
Day 2 取得網址！ 目前字典長度: 363  
注意：「 //div[@data-swiper-slide-index]  +  style 」的組合失敗。  
Day 3 取得網址！ 目前字典長度: 475  
下載結束

### 7. 關閉瀏覽器 / close the browser

In [ ]:
driver.close()